Este codigo para extraer los datos de user reviews en un data frame de pandas
archivo: "C:\Users\Usuario\Henry\PI01MLOPnuevo\user_reviews.json.gz"

In [ ]:
pip install textblob 

In [2]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import csv
import time
import string
from textblob import TextBlob

In [3]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(user_reviews):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(user_reviews, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

ast.literal_eval es necesario porque los datos en el archivo parecen estar en formato JSON pero almacenados como cadenas en lugar de objetos de Python. literal_eval es más seguro que eval ya que solo evalúa expresiones literales de Python y evita la ejecución de código malicioso.

In [4]:
datos = []
for i  in gzip.open('user_reviews.json.gz'):
    datos.append(ast.literal_eval(i.decode('utf-8')))

df_user_reviews= pd.DataFrame(datos)

df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [5]:
tamaño_df_user_reviews=df_user_reviews.shape
tamaño_df_user_reviews

(25799, 3)

In [ ]:
df_user_reviews.head()

In [ ]:
# Lo primero es explotar la columna reviews y posterior se desanida normalizandola

df_user_reviews_exploded = df_user_reviews.explode('reviews').dropna()

df_user_reviews_exploded.head()

In [ ]:
df_user_reviews_exploded.info()

In [ ]:
df_user_reviews_exploded['reviews'].astype(str)

In [ ]:
df_user_reviews_exploded.head()
#El tipo sigue siendo object pero continuo

In [11]:
tamaño_df_user_reviews_exploded = df_user_reviews_exploded.shape
tamaño_df_user_reviews_exploded


(59305, 3)

La función normalize se utiliza para aplanar datos JSON anidados en un DataFrame. Cuando se trabaja con datos JSON que tienen estructuras anidadas (como diccionarios dentro de diccionarios), json_normalize() ayuda a convertir esos datos en un formato tabular que es más fácil de trabajar con pandas.

In [12]:
df_user_reviews_desanidado = pd.json_normalize(df_user_reviews_exploded['reviews'])

In [13]:
df_user_reviews_desanidado.shape
#se mantiene el mismo tamaño

(59305, 7)

In [ ]:
df_user_reviews_desanidado.head()
#la clase es pandas pero en esa clase no hay metodos concomo drop, toca pasarlo a str

In [15]:
#borro columna last_edited 

df_user_reviews_desanidado = df_user_reviews_desanidado.drop('last_edited', axis=1)

In [16]:
df_user_reviews_desanidado.reset_index(inplace=True)


In [ ]:
df_user_reviews_desanidado.head()

In [ ]:
df_user_reviews_desanidado

In [ ]:
# De una vez borro nullen el df sin desanidar , pero no disminuyo en nada, así que borro los duplicados

In [19]:

df_user_reviews = df_user_reviews.dropna()


In [20]:
#concateno el df inicial con el exploded
df_user_reviews_sentimiento = pd.concat([df_user_reviews, df_user_reviews_desanidado], axis=1)


In [ ]:
df_user_reviews_sentimiento.head()

In [ ]:
df_user_reviews_sentimiento

In [24]:
# borro columa funny que no tiene sentido, 

df_user_reviews_sentimiento = df_user_reviews_sentimiento.drop('funny', axis=1)

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' 
aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral 
y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos 
de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    # Devuelve la polaridad del sentimiento (-1 a 1)
    return analysis.sentiment.polarity

    Aplica la función a la columna relevante de tu DataFrame:
    df['Sentimiento'] = df['Texto'].apply(analizar_sentimiento)

In [ ]:
df_user_reviews_sentimiento['review']

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    # Devuelve la polaridad del sentimiento (-1 a 1)
    return analysis.sentiment.polarity

    Aplica la función a la columna relevante de tu DataFrame:
    df['Sentimiento'] = df['Texto'].apply(analizar_sentimiento)

In [26]:
def analisis_sentimiento(Texto):
    analisis = TextBlob(Texto)
    if analisis.polarity>0:
        return 2
    elif analisis.polarity<0:
        return 0
    elif analisis.polarity==0:
        return 1

In [58]:
# Corro la función analisis_sentimiento que se creo sobe la columna review del data frame y lo reemplazo

In [ ]:
df_user_reviews_sentimiento['review'] = df_user_reviews_sentimiento['review'].apply(f_user_reviews_sentimiento)

In [28]:
# Renombrar la columna
df_user_reviews_sentimiento = df_user_reviews_sentimiento.rename(columns = {'review':'sentiment_analysis'})

In [ ]:
df_user_reviews_sentimiento

In [30]:
df_user_reviews_sentimiento.to_json('df_user_reviews_sentimiento.json',orient='records')